In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px


In [16]:
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.*")

In [17]:

# Setting the seed for the random number generator for reproducibility
np.random.seed(13)

# Step 1: Read the data from the CSV file
file_path = '/Users/elenabolokhonova/visualization/02_activities/assignments/assignment_4_materials/data/Fire Incidents Data.csv'
data_row = pd.read_csv(file_path)

# Review the data
data_row.head()

/var/folders/mn/hzb_sstd3t9f6xyvy2fry_5m0000gn/T/ipykernel_5224/423921863.py:6: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data_row = pd.read_csv(file_path)


_id                      Area_of_Origin          Building_Status  \
0  3199061                    81 - Engine Area                      NaN   
1  3199062  75 - Trash, rubbish area (outside)                      NaN   
2  3199063                                 NaN                      NaN   
3  3199064  75 - Trash, rubbish area (outside)  01 - Normal (no change)   
4  3199065                                 NaN                      NaN   

                Business_Impact  Civilian_Casualties  \
0                           NaN                  0.0   
1                           NaN                  0.0   
2                           NaN                  0.0   
3  1 - No business interruption                  0.0   
4                           NaN                  0.0   

   Count_of_Persons_Rescued  Estimated_Dollar_Loss  \
0                       0.0                15000.0   
1                       0.0                   50.0   
2                       0.0                    NaN   
3                       0.0                    0.0   
4                       0.0                    NaN   

   Estimated_Number_Of_Persons_Displaced  Exposures  \
0                                    NaN        NaN   
1                                    NaN        NaN   
2                                    NaN        NaN   
3                                    0.0        NaN   
4                                    NaN        NaN   

  Ext_agent_app_or_defer_time  ...  \
0         2018-02-24T21:12:00  ...   
1         2018-02-24T21:29:42  ...   
2                         NaN  ...   
3         2018-02-25T14:19:25  ...   
4                         NaN  ...   

            Smoke_Alarm_at_Fire_Origin_Alarm_Failure  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  98 - Not applicable: Alarm operated OR presenc...   
4                                                NaN   

  Smoke_Alarm_at_Fire_Origin_Alarm_Type  \
0                                   NaN   
1                                   NaN   
2                                   NaN   
3                 9 - Type undetermined   
4                                   NaN   

  Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation  \
0                                                NaN              
1                                                NaN              
2                                                NaN              
3   8 - Not applicable: No alarm, no persons present              
4                                                NaN              

        Smoke_Spread                        Sprinkler_System_Operation  \
0                NaN                                               NaN   
1                NaN                                               NaN   
2                NaN                                               NaN   
3  99 - Undetermined  8 - Not applicable - no sprinkler system present   
4                NaN                                               NaN   

  Sprinkler_System_Presence  \
0                       NaN   
1                       NaN   
2                       NaN   
3          9 - Undetermined   
4                       NaN   

                           Status_of_Fire_On_Arrival       TFS_Alarm_Time  \
0  7 - Fully involved (total structure, vehicle, ...  2018-02-24T21:04:29   
1              2 - Fire with no evidence from street  2018-02-24T21:24:43   
2                                                NaN  2018-02-25T13:29:59   
3  3 - Fire with smoke showing only - including v...  2018-02-25T14:13:39   
4                                                NaN  2018-02-25T18:20:43   

      TFS_Arrival_Time  TFS_Firefighter_Casualties  
0  2018-02-24T21:10:11                         0.0  
1  2018-02-24T21:29:31                         0.0  
2  2018-02-25T13:36:49                         0.0  
3  2018-02-25T14:18:07      

In [18]:
# Initial data cleaning: create the data_cleaned DataFrame
data_cleaned = data_row.copy()

# Drop rows with missing 'Area_of_Origin', 'Estimated_Dollar_Loss', and other critical columns
data_cleaned = data_cleaned.dropna(subset=['Area_of_Origin', 'Estimated_Dollar_Loss', 'Building_Status'])

# Print remaining missing values after dropping rows with missing critical values
print("Remaining missing values after dropping rows with missing critical values:")
print(data_cleaned.isnull().sum())

Remaining missing values after dropping rows with missing critical values:
_id                                                                  0
Area_of_Origin                                                       0
Building_Status                                                      0
Business_Impact                                                      4
Civilian_Casualties                                               2430
Count_of_Persons_Rescued                                             0
Estimated_Dollar_Loss                                                0
Estimated_Number_Of_Persons_Displaced                                3
Exposures                                                        13954
Ext_agent_app_or_defer_time                                          1
Extent_Of_Fire                                                       4
Final_Incident_Type                                                  0
Fire_Alarm_System_Impact_on_Evacuation                               3
Fi

In [19]:
# Fill missing numerical values with median
numerical_cols_to_impute = ['Civilian_Casualties', 'Estimated_Number_Of_Persons_Displaced']
for col in numerical_cols_to_impute:
    data_cleaned[col].fillna(data_cleaned[col].median(), inplace=True)

# Fill missing date columns with the most frequent date
data_cleaned['Fire_Under_Control_Time'] = pd.to_datetime(data_cleaned['Fire_Under_Control_Time'])
most_frequent_date = data_cleaned['Fire_Under_Control_Time'].mode()[0]
data_cleaned['Fire_Under_Control_Time'].fillna(most_frequent_date, inplace=True)

# Fill missing categorical values with mode
categorical_cols_to_impute = ['Business_Impact', 'Ext_agent_app_or_defer_time', 'Extent_Of_Fire', 
                              'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation', 
                              'Fire_Alarm_System_Presence', 'Incident_Ward', 'Initial_CAD_Event_Type', 
                              'Level_Of_Origin']
for col in categorical_cols_to_impute:
    data_cleaned[col].fillna(data_cleaned[col].mode()[0], inplace=True)

print("Remaining missing values after imputation:")
print(data_cleaned.isnull().sum())


Remaining missing values after imputation:
_id                                                                  0
Area_of_Origin                                                       0
Building_Status                                                      0
Business_Impact                                                      0
Civilian_Casualties                                                  0
Count_of_Persons_Rescued                                             0
Estimated_Dollar_Loss                                                0
Estimated_Number_Of_Persons_Displaced                                0
Exposures                                                        13954
Ext_agent_app_or_defer_time                                          0
Extent_Of_Fire                                                       0
Final_Incident_Type                                                  0
Fire_Alarm_System_Impact_on_Evacuation                               0
Fire_Alarm_System_Operation       

In [22]:
# Save the cleaned data to a new CSV file
cleaned_file_path = '/Users/elenabolokhonova/visualization/02_activities/assignments/assignment_4_materials/data/Fire_Incidents_Cleaned.csv'
data_cleaned.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to /Users/elenabolokhonova/visualization/02_activities/assignments/assignment_4_materials/data/Fire_Incidents_Cleaned.csv


In [23]:
# Example cleaned dataset with added columns for demonstration
data = {
    'Area_of_Origin': [
        'Cooking Area or Kitchen', 'Porch or Balcony', 'Sleeping Area or Bedroom',
        'Trash, Rubbish Storage', 'Living Area', 'Garage', 'Laundry Area',
        'Washroom or Bathroom', 'Roof', 'Process Manufacturing'
    ],
    'Frequency': [4000, 1500, 1300, 1100, 920, 600, 540, 470, 370, 300],
    'Estimated_Dollar_Loss': [100000, 50000, 40000, 30000, 20000, 15000, 12000, 10000, 8000, 5000],
    'Civilian_Casualties': [5, 3, 4, 2, 1, 0, 2, 1, 0, 0]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Define the color scale you want to use
color_scale = 'Plasma'  # Change to any other available color scale

# Create and save individual frames
filenames = []
for i in range(10):
    # Update the data or the layout for each frame
    fig = px.bar(
        df, 
        x='Area_of_Origin', 
        y='Frequency', 
        hover_data=['Estimated_Dollar_Loss', 'Civilian_Casualties'],  # Additional hover data
        color='Estimated_Dollar_Loss',  # Color scale based on Estimated Dollar Loss
        labels={'Frequency': 'Frequency of Incidents', 'Estimated_Dollar_Loss': 'Estimated Dollar Loss', 'Civilian_Casualties': 'Civilian Casualties'},
        height=500,  # Adjusted height for smaller figure
        width=800,  # Adjusted width for smaller figure
        color_continuous_scale=color_scale  # Use the chosen color scale
    )
    
    fig.update_layout(
        title={
            'text': 'Top 10 Most Frequent Areas of Origin for Fire Incidents responded by "Toronto Fire Service"',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {
                'size': 18,
                'family': 'Arial, sans-serif',
                'color': 'purple'
            }
        },
        xaxis_title='Area of Origin',
        xaxis_title_font=dict(size=16, family='Arial, sans-serif', color='black'),  # Adjusted axis title font size
        yaxis_title='Frequency',
        yaxis_title_font=dict(size=16, family='Arial, sans-serif', color='black'),  # Adjusted axis title font size
        xaxis_tickfont=dict(size=12, family='Arial, sans-serif', color='black'),  # Adjusted tick font size
        yaxis_tickfont=dict(size=12, family='Arial, sans-serif', color='black'),  # Adjusted tick font size
        xaxis_tickangle=-45,  # Rotate x-axis labels
        plot_bgcolor='white',
        bargap=0.2,  # Gap between bars
        margin=dict(l=50, r=50, t=150, b=150),  # Adjusted margins for smaller figure
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',  # Set x-axis grid color to light grey
            gridwidth=1,  # Set x-axis grid line width
            griddash='solid'  # Set x-axis grid line style
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',  # Set y-axis grid color to light grey
            gridwidth=1,  # Set y-axis grid line width
            griddash='solid'  # Set y-axis grid line style
        )
    )
    


# Show the interactive plot
fig.show()
